In [28]:
from IPython.display import display, HTML,clear_output
import random as rnd
import time

In [56]:
class SVGCanvas:
    def __init__(self,max_number,size):
        self.width = max_number*size
        self.height = max_number*size
        self.size = size
        self.max_number = max_number
        self.html_s =""
    
    def clear(self):
        self.html_s = '<svg width="'+str(self.width)+'" height="'+str(self.height)+'"style="border:1px solid black">'

    def circle(self,x_pos,y_pos,color):
        x_pos = x_pos * self.size
        y_pos = y_pos * self.size
        str_int_x = str(int(x_pos))
        str_int_y = str(int(y_pos))
        r = 1/(self.size*2)
        str_int_x = str(int((x_pos*self.width)+(r*self.width)))
        str_int_y = str(int((y_pos*self.height)+(r*self.width)))
        str_int_r = str(int(r))
        self.html_s +='<circle cx="'+str_int_x+'" cy="'+str_int_y+'" r="'+str_int_r+'" fill="'+color+'"/>'
    
    def rect(self,x_pos,y_pos,color):
        x_pos = x_pos * self.size
        y_pos = y_pos * self.size
        str_int_x = str(int(x_pos))
        str_int_y = str(int(y_pos))
        str_int_width = str(int(self.size))
        str_int_height = str(int(self.size))
        self.html_s +='<rect x="'+str_int_x+'" y="'+str_int_y+'" width="'+str_int_width+'" height="'+str_int_height+'" fill="'+color+'"/>'

    def getCanvas(self):
        self.html_s += "</svg>"
        return HTML(self.html_s)

In [57]:
class Counter:
    def __init__(self):
        self.tnow =0
        self.tnext = 1
        
    def update(self):
        # This switches the 0 1 state of now and next
        self.tnow = (self.tnow+1) % 2
        self.tnext = (self.tnext+1) % 2

In [58]:
class Cell:
    max_resource = 30
    def __init__(self,x_pos,y_pos,svg,counter):
        self.neighbours = []
        self.number_of_neighbours=-1
        self.x_pos = x_pos
        self.y_pos = y_pos
        self.svg = svg
        self.resource =0.0
        self.counter = counter
        self.occupant = None
        
    def addNeighbour(self,cell):
        self.neighbours.append(cell)
        self.number_of_neighbours+=1
    
    def randomNeighbour(self):
        return rnd.choice(self.neighbours)
    
    
    @property
    def resource_rgb(self):
        return str(int(255-(self.resource*255)/Cell.max_resource))
    
    @property
    def empty(self):
        return self.occupant == None
    
    def draw(self):
        self.svg.rect(self.x_pos,self.y_pos,'rgb(255,'+self.resource_rgb+','+self.resource_rgb+')')
        if self.occupant !=None:
            self.svg.circle(self.x_pos,self.y_pos,'blue')
    
    def paint(self,color):
        self.svg.rect(self.x_pos,self.y_pos,color)

    def printPos(self):
        print("x_pos:",self.x_pos)
        print("y_pos:",self.y_pos)

    def print(self):
        self.printPos()
        print()
        for cell in self.neighbours:
            cell.printPos()
        print()

    def search_res(self,depth):
        print("here",depth)
        max = [self.resource,0,self]
        off_set = rnd.randint(0,4)
        for i in range(4):
            c = (i+off_set)%4
            cell = self
            for j in range(depth):
                cell = cell.neighbours[c]
                res = cell.resource
                if res> max[0]:
                    max=[res,j,cell]
                print(res,c,off_set,i)
        print()
        print(max)
        print(self.x_pos,self.y_pos)

                

In [59]:
class Agent:
    def __init__(self,home,counter):
        self.home = home
        self.home.occupant = self
        self.counter = counter
    
    def randomMove(self):
        move_to = self.home.randomNeighbour()
        if move_to.empty:
            self.home.occupant = None
            self.home = move_to
            self.home.occupant = self
    

In [61]:
class Experiment:
    def __init__(self,size):        
        self.cells = []
        self.agents =[]
        self.size = size
        self.total_cells = size*size
        self.svg = SVGCanvas(size,15)
        

    def setUp(self,agents):
        self.cells = []
        self.agents = []
        self.counter = Counter()
        
        for i in range(self.size*self.size):
            n_cell = Cell((i%self.size),int(i/self.size),self.svg,self.counter)
            n_cell.resource = (i%self.size)+int(i/self.size)
            self.cells.append(n_cell)
        for cell in self.cells:
            for d_pos in[(-1,0),(0,-1),(1,0),(0,1)]:
                x=self.bounds(cell.x_pos+d_pos[0])
                y=self.bounds(cell.y_pos+d_pos[1])
                pos = y*self.size+x
                cell.addNeighbour(self.cells[pos])
        for _ in range(agents):
            home = rnd.choice(self.cells)
            if home.empty:
                agent = Agent(home,self.counter)
                self.agents.append(agent)
        
    def bounds(self,i):
        if i<0:
            return self.size + i 
        if i>=self.size:
            return i-self.size
        return i
        
    def iterate(self):
        rnd.shuffle(self.agents)
        for agent in self.agents:
            agent.randomMove()
        self.draw()
        
    def draw(self):
        self.svg.clear()
        for cell in self.cells:
            cell.draw()
        clear_output(wait=True)
        display(self.svg.getCanvas())


In [62]:
experiment = Experiment(16)
experiment.setUp(0)
experiment.svg.clear()
for m_cell in experiment.cells:
     m_cell.draw()
#     for cell in m_cell.neighbours:
#        cell.paint("blue")
clear_output(wait=True)
display(experiment.svg.getCanvas())
#     print(m_cell.x_pos,m_cell.y_pos)
#     time.sleep(0.1)

In [63]:
experiment = Experiment(16)
experiment.setUp(0)
m_cell = experiment.cells[50]
print(m_cell.x_pos,m_cell.y_pos)
m_cell.search_res(2)

2 3
here 2
6 3 3 0
7 3 3 0
4 0 3 1
3 0 3 1
4 1 3 2
3 1 3 2
6 2 3 3
7 2 3 3

[7, 1, <__main__.Cell object at 0x7f9f102ae5e0>]
2 3
